In [2]:
pip install dash


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install dash-leaflet

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install leafmap

   ---------------------------------------- 0.0/567.9 kB ? eta -:--:--
   ------------------ --------------------- 262.1/567.9 kB ? eta -:--:--
   ---------------------------------------- 567.9/567.9 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.2 MB 1.5 MB/s eta 0:00:01
   ------------------------- -------------- 0.8/1.2 MB 1.4 MB/s eta 0:00:01
   --------------------------------- ------ 1.0/1.2 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.3 MB 1.8 MB/s eta 0:00:06
   -- ------------------------------------- 0.8/11.3 MB 1.6 MB/s eta 0:00:07
   --- ---------------------------------

Useful links:
https://www.dash-leaflet.com/components/controls/edit_control

Definition of useful libraries

In [1]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign

import branca.colormap as cm

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from dash.exceptions import PreventUpdate


Definition of useful functions

In [2]:
# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# get_province_shape returns a GeoDataFrame containing the geometries of the provinces
   
def get_province_shape():
    t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf

# list_sensors_and_pollutants gets the list of sensor and relative pollutants for a given station

def list_sensors_and_pollutants(list_stations):
    t=requests.post(url="http://127.0.0.1:5000/api/sensors_and_pollutants", json={"var_id_stazione": list_stations}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data

# get_station_location returns a GeoDataFrame containing the points of the stations

def get_station_location():
    t=requests.get(url="http://127.0.0.1:5000/api/station_location")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

############################################# FUNCTION TO POPULATE THE DROPDOWN #############################################

# pollutant_dropdown populates the dropdown with the pollutatnts that have at least one recorded value

def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

############################################# FUNCTION FOR MAP VISUALIZATION #############################################

# df_to_dash_table converts a DataFrame to a Dash HTML table for display in the popup of EU_DV_1

def df_to_dash_table(df):
    return html.Table([
        html.Thead(html.Tr([html.Th(col) for col in df.columns])),
        html.Tbody([
            html.Tr([html.Td(df.iloc[i][col]) for col in df.columns]) for i in range(len(df))
        ])
    ], style={"maxWidth": "300px", "fontSize": "12px"})

Definition of variables for visualization

In [3]:
# prov is a GeoDataFrame containing the geometries of the provinces
prov=get_province_shape()

# geojson is a dictionary containing the geometry of the provinces in GeoJSON format
geojson = mapping(prov.geometry_province)

# the two following variables are used to create a green and a red icon marker, later used in the map

#https://github.com/pointhi/leaflet-color-markers
custom_icon_green = dict(
    iconUrl= 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-green.png',
    shadowUrl= 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
    iconSize= [25, 41],
    iconAnchor= [12, 41],
    popupAnchor= [1, -34],
    shadowSize= [41, 41]
)
custom_icon_red = dict(
    iconUrl= 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png',
    shadowUrl= 'https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/images/marker-shadow.png',
    iconSize= [25, 41],
    iconAnchor= [12, 41],
    popupAnchor= [1, -34],
    shadowSize= [41, 41]
)

# How to render geojson.
point_to_layer = assign("""function(feature, latlng, context){
    const p = feature.properties;
    if(p.type === 'circlemarker'){return L.circleMarker(latlng, radius=p._radius)}
    if(p.type === 'circle'){return L.circle(latlng, radius=p._mRadius)}
    return L.marker(latlng);
}""")


In [4]:
# Silvia

# Get data for initial pollutant (DV 10)
def get_data_10(pollutant):
    data = {"var_pollutant": pollutant}
    t = requests.post(url="http://127.0.0.1:5000/api/DV_10", json=data)
    data = read_response(t)
    # to visualize the response, we can conver the data to a pandas geodataframe
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)

    gdf = gdf.dropna(subset=['quota']) # Drop rows where 'quota' is NaN

    gdf['month'] = pd.to_datetime(gdf['month'], errors='coerce')
    gdf['month'] = gdf['month'].dt.strftime('%b %Y')
    return gdf

# Get data for initial pollutant (DV 11)
def get_data_11(pollutant):
    data = {"var_pollutant": pollutant}
    t = requests.post(url="http://127.0.0.1:5000/api/DV_11", json=data)
    data = read_response(t)
    df = pd.DataFrame(data)
    df['month'] = pd.to_datetime(df['month'], errors='coerce')
    df['month'] = df['month'].dt.strftime('%b %Y')
    return df



In [5]:
# contour lines file (DV 10)
import json
with open("../DATA/contour_lines.geojson", "r") as f:
    geojson_data = json.load(f)

#ADD colour to geojson based on elevation

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def get_color(elev, min_elev=500, max_elev=3000):
    # Define a color map transitioning smoothly across elevations
    cmap = mcolors.LinearSegmentedColormap.from_list("terrain", [
        (0.0, "lawngreen"),   # Low elevation (green)
        (0.3, "yellow"),  # Mid elevation (yellow)
        (0.6, "orange"),  # High elevation (orange)
        (1.0, "brown")      # Highest elevation (red)
    ])

    # Normalize elevation to [0, 1] range
    ratio = (elev - min_elev) / (max_elev - min_elev)
    ratio = min(1, max(0, ratio))  # Clamp between 0 and 1

    # Convert colormap color to HEX format
    rgb = cmap(ratio)[:3]  # Get RGB values
    return f"#{int(rgb[0] * 255):02x}{int(rgb[1] * 255):02x}{int(rgb[2] * 255):02x}"

# Supponiamo tu abbia geojson_data come dizionario o FeatureCollection
for feature in geojson_data["features"]:
    elev = feature["properties"].get("ELEV", 1000)
    color = get_color(elev)
    feature["properties"]["color"] = color

print(geojson_data["features"][0])  # check the features structure

{'type': 'Feature', 'properties': {'fid': 1, 'ELEV': 2200.0, 'color': '#ec8b08'}, 'geometry': {'type': 'LineString', 'coordinates': [[10.237773054012042, 46.63439685859683], [10.237771002440088, 46.63430688921537], [10.237555801129059, 46.63427234207511], [10.237425411671927, 46.63428339632108]]}}


In [6]:
#DV 10 functions --------------------------------------------------
# Style function for contour lines 

style = assign("""
function(feature) {
    return {
        color: feature.properties.color || "blue",
        weight: 1,
        opacity: 0.8
    };
}
""")

# DV 11 functions --------------------------------------------------
def get_years(df):
    return sorted(df["month"].str[-4:].unique())

def get_months(df, year):
    months = sorted(df[df["month"].str.contains(str(year[0]))]["month"].str[:3].unique(),
                    # key to get the months in order
                    key=lambda m: ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].index(m))
    return ["All"] + months

In [ ]:
# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([

    html.H1('Bugs_project: Air quality analysis'),
    html.P('Description of the dashboard functionalities'),

    ############################################# User selection #############################################
    html.Div([
        html.H2('User selection'),
        html.P('Select the type of user you are:'),
        dcc.RadioItems(
            id='user-selection',
            options=[
                {'label': 'Normal User', 'value': 1},
                {'label': 'Expert User', 'value': 2}
            ],
            value=0,
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        ),
    ], id='user', hidden=False, style={'padding': 20, 'margin':2, 'border': '1px solid #ccc'}),

    ############################################# DV_5 #############################################
    html.Div([
        html.Div([
        html.H2('DV_5 Single Sensor Time Series'),
        html.P('Start by selecting a pollutant, then select one among all the sensors that measures that pollutant, and then visualize in the graph the time series of the monthly average of the data measured by that sensor'),

        #Dropdown to select the pollutant
            dcc.Dropdown(
            id='pollutant-dropdown_dv_5',
            options=pollutant_dropdown(),
            placeholder="Select a pollutant",
            style={'margin-top':20}
        ),

        #Dropdown to select the sensor
            dcc.Dropdown(
            id='sensor-dropdown_dv_5',
            placeholder="Select a sensor",
            style={'margin-top':20}
        ),

        #Button to activate the graph
        html.Div([
            html.Button('Visualize/update time series', id='time_series_dv_5', n_clicks=0, style={'margin-top': 20, 'margin-bottom':20, 'padding': '6px 12px', 'fontSize': '15px',})
        ], style={'textAlign': 'center'}),

        #Visualiza the graph
        # Loading wrapper for the map
        dcc.Loading(
            type="dot",  
            children=dcc.Graph(id='time_series_single_sensor_dv_5'),
        )
        
        ], style={'padding': 20, 'margin':2, 'border': '1px solid #ccc'}),

        ############################################# DV_6 #############################################

        html.Div([
        html.H2('DV_6 Location of all the sensors of a selected pollutant'),
        html.P('Choose a pollutant, and then visualize on a map the location af all the sensors of that pollutant. The sensor will appear red if it\'s still active, and will appear gray if it is not active anymore. Clicking on the icon of a sensor the user can visualiza the id of the sensor, the name of the station in which the sensor is located and date in which the sensor was deactivated, if it is not active anymore'),
        
        #Dropdown to select the pollutant
            dcc.Dropdown(
            id='all-pollutant-dropdown_dv_6',
            options=all_pollutant_dropdown(),
            placeholder="Select a pollutant",
            style={'margin-top':20}
        ),

        #Button to activate the map
        html.Div([
            html.Button('Visualize/update map', id='button_dv_6', n_clicks=0, style={'margin-top': 20, 'margin-bottom':20, 'padding': '6px 12px', 'fontSize': '15px',})
        ], style={'textAlign': 'center'}),

        #Visualization of the map
        dcc.Loading(
            type="dot",  
            children=dl.Map(
                children=[
                    dl.TileLayer(),  # base layer
                    dl.LayersControl([dl.Overlay(
                            dl.GeoJSON(
                                data=geojson,
                                id="province_dv_6",
                                zoomToBounds=True,
                                options=dict(style=dict(color="blue", weight=1))
                            ),
                            name="Province",
                            checked=True
                        ),
                        dl.Overlay(dl.LayerGroup(id="active_dv_6", children=[]), name="Active sensors", checked=True),
                        dl.Overlay(dl.LayerGroup(id="inactive_dv_6", children=[]), name="Inactive sensors", checked=True),],id="lc_dv_6")
                ],
                center=[45.64, 9.60],
                zoom=8,
                style={"height": "50vh"}
            )
        )
        
        ], style={'padding': 20, 'margin':2, 'border': '1px solid #ccc'}),

    ############################################# DV_10 #############################################

        html.Div([
            html.H3("DV 10 - Correlation map between monthly average of pollutant and station height"),
            html.Label("Pollutant:"),
            dcc.Dropdown(
                id='pollutant-dropdown_dv_10_11',
                options=pollutant_dropdown(),
                style={'margin-top': 20, 'margin-bottom': '20'},
            ),
            html.Div([
                html.Button('Visualize/update elevation map', id='el_map_dv_10', n_clicks=0, style={'margin-top': 20, 'margin-bottom': 20, 'padding': '6px 12px', 'fontSize': '15px'})
            ], style={'textAlign': 'center'}),

            # Loading wrapper for the map
            dcc.Loading(
                type="dot",  
                children=dl.Map(
                    id="map",
                    center=[45.64, 9.60],
                    zoom=7,
                    children=[dl.TileLayer(), dl.LayersControl(id="layers-control_dv_10")],
                    style={"opacity": 0.7, "height": "50vh"},
                )
            )
        ]),

        html.Hr(style={"border": "1px solid lightgreen", "opacity": 0.7, "margin": "30px 0"}),  # Divider

        ############################################# DV_11 #############################################
        
        html.Div([
            html.H3("DV 11 - Correlation between monthly average of pollutant and station height"),
            html.Label("Select between available years of the pollutant chosen in DV 10:"),
            dcc.Slider(id='year-slider', step=1.0),
            dcc.Dropdown(id='month-dropdown'),
            html.Div([
                html.Button('Visualize/update plot', id='el_plot_dv_11', n_clicks=0, style={'margin-top': 20, 'margin-bottom': 20, 'padding': '6px 12px', 'fontSize': '15px'})
            ], style={'textAlign': 'center'}),

            # Loading wrapper for the scatter plot
            dcc.Loading(
                type="dot",
                children=dcc.Graph(id='scatter-plot', style={'margin-top': '20px'})
            )
        ]),


    ], id='normal_user'),



    ############################################# EU_DV_1 #############################################
    html.Div([
        html.Div([
        html.H2('EU_DV_1 Expert user'),
        html.P('In this section an expert user can select a region of interest on the map using the buttons on the left. Once the region of interest has been selected the user can visualize on the map the stations inside the selected area. The station will appear red if it contains at least one active sensor, otherwise it will be gray. By clicking on the icon of a station the user can visualize which sensors are inside the station, the pollutant that they measure and the date in which they have been deactivated, if they are not active anymore'),
            
        # Display the map in which the user can select a region of interest
        dl.Map(
            children=[
            dl.TileLayer(),  # base layer
            dl.FeatureGroup([
                dl.EditControl(id="edit_control_eu_dv_1",draw={
                    "circle": False,         
                    "circlemarker": False,   
                    "rectangle": True,
                    "polygon": True,
                    "marker": False,
                    "polyline": False,
                    },),
            ]),
            dl.LayersControl(dl.Overlay(dl.GeoJSON(
                        data=geojson,
                        id="lombardia_eu_dv_1",
                        zoomToBounds=True,
                        #interactive=False,
                        options=dict(style=dict(color="blue", weight=1))
                    ), name="Lombardia",checked=True ))],
            center=[45.64, 9.60],
            zoom=8,
            style={"height": "50vh"}
        ),
        
        #Button the reset the selection of the user
        html.Div([
            html.Button("Remove -> Clear all", id="clear_all_eu_dv_1", style={'margin-top': 20, 'margin-bottom':20, 'padding': '6px 12px', 'fontSize': '15px',}),
        ], style={'textAlign': 'center'}),
        

        html.P("These are the stations inside the selected region of interes, if the region is empty it means that no stations are inside the select area"),

        # Display the stations inside the region of interest on the new map
        dl.Map(
            children=[
                dl.TileLayer(),  # base layer
                dl.GeoJSON(id="geojson_eu_dv_1", pointToLayer=point_to_layer, zoomToBounds=True),
                dl.LayersControl([dl.Overlay(
                        dl.GeoJSON(
                            data=geojson,
                            id="province_eu_dv_1",
                            zoomToBounds=True,
                            options=dict(style=dict(color="blue", weight=1))
                        ),
                        name="Province",
                        checked=True
                    ),
                    dl.Overlay(dl.LayerGroup(id="active_eu_dv_1", children=[]), name="Active sensors", checked=True),
                    dl.Overlay(dl.LayerGroup(id="inactive_eu_dv_1", children=[]), name="Inactive sensors", checked=True),],id="lc_eu_dv_1")
            ],
            style={"height": "50vh"},
            center=[45.64, 9.60],
            zoom=7,id="final"),
        ], style={'padding': 20, 'margin':2, 'border': '1px solid #ccc'}),
    ], id='expert_user'),
])

############################################# Callbacks user #############################################
@app.callback(
    [Output('normal_user', 'hidden'),
    Output('expert_user', 'hidden'),
    Output('user', 'hidden'),
    ],
    Input('user-selection','value')
)
def user_selection(selected_user):
    if selected_user == 0:
        return [True, True, False]
    elif selected_user == 1:
        return [False, True, True]  
    else:
        return [False, False, True]
    
############################################# Callbacks DV_5 #############################################

#Callback to get the sensor list after the user selects a pollutant
@app.callback(
    Output('sensor-dropdown_dv_5', 'options'),
    Input('pollutant-dropdown_dv_5','value')
)
def sensor_dropdown(selected_pollutant):
    if not selected_pollutant:
        options = []
        return options
    t=requests.post(url="http://127.0.0.1:5000/api/sensor", json={"var_pollutant": selected_pollutant})
    list_sensors = read_response(t)
    options = [{'label': f'{sensor['nome_stazione']} - {sensor['id_sensore']}', 'value': sensor['id_sensore']} for sensor in list_sensors]
    return options

#Callback to print the graph of of requirement 5: time series of a single sensor
@app.callback(
    Output('time_series_single_sensor_dv_5','figure'),
    Input('time_series_dv_5', 'n_clicks'),
    State('pollutant-dropdown_dv_5','value'),
    State('sensor-dropdown_dv_5','value')
)
def time_series(n_clicks, selected_pollutant, sensor_id):
    if n_clicks > 0:
        if not sensor_id:
            return px.line(title='Select a pollutant and a sensor, then press the button to visualize the time series')
        data = {"var_sensor_id": sensor_id, "var_pollutant": selected_pollutant} #Right now data is a dictionary
        # send the request to the server, we use post because we are sending data to the server
        t=requests.post(url="http://127.0.0.1:5000/api/DV_5", json=data) #json= data will convert the dictionary to a json object and send it to the server
        data = read_response(t) 
        df = pd.DataFrame(data)
        if df.empty:
            return px.line(title='No data available for the selected sensor and pollutant')
        df['month'] = pd.to_datetime(df['month'], errors='coerce') #keeps just year and month for the date
        df['month'] = df['month'].dt.strftime('%b %Y')
        unit=get_measurement_unit(selected_pollutant) #gets the unit of measurement for a pollutant
        line_chart = px.line(df, x='month', y='monthly_avg', title=f'Monthly average for {selected_pollutant} of sensor {sensor_id} in {unit}')
        return line_chart
    return px.line(title='Select a pollutant and a sensor, then press the button to visualize the time series')

############################################# Callbacks DV_6 #############################################

#Callback to print the map of requirement 6
@app.callback(
    [Output('active_dv_6', 'children'),
     Output('inactive_dv_6','children')],
    Input('button_dv_6', 'n_clicks'),
    State('all-pollutant-dropdown_dv_6','value')
)
def map_sensors_location(n_clicks, selected_pollutant):

    if n_clicks == 0:
      return [[],[]]
    
    if n_clicks > 0 and not selected_pollutant:
        return [dl.Popup(
                    position=[45.64, 9.60],  # Central position
                    children=html.Div(
                        "Select a pollutant to visualize the map",
                        style={"backgroundColor": "red", "color": "white", "padding": "10px", "borderRadius": "5px"}
                    ))
                ,[]
            ]

    if n_clicks > 0:
        #if not selected_pollutant:
         #   return [[],[]]
    
        #get the data
        data = {"var_pollutant" : selected_pollutant}

        # send the request to the server, we use post because we are sending data to the server
        t=requests.post(url="http://127.0.0.1:5000/api/DV_6", json=data) #json= data will convert the dictionary to a json object and send it to the server

        data = read_response(t)
        gdf = gpd.GeoDataFrame(data)
        gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
        gdf.set_geometry('geometry', inplace=True)
        
        #create the markers
        active = gdf[gdf['data_stop'].isnull()]
        inactive = gdf[gdf['data_stop'].notnull()]
        inactive['data_stop'] = pd.to_datetime(inactive['data_stop'], errors='coerce') #keeps just year and month for the date
        inactive['data_stop'] = inactive['data_stop'].dt.strftime('%b %Y')

        # Crea Marker attivi
        active_markers = [
            dl.Marker(
                position=[row.geometry.y, row.geometry.x],
                children=dl.Popup([
                    html.B(f"{row['id_sensore']} - {row['nome_stazione']}"),
                    html.Br(),
                    "Active"
                ]),
                icon=custom_icon_green
            ) for _, row in active.iterrows()
        ]

        # Marker inattivi
        inactive_markers = [
            dl.Marker(
                position=[row.geometry.y, row.geometry.x],
                children=dl.Popup([
                    html.B(f"{row['id_sensore']} - {row['nome_stazione']}"),
                    html.Br(),
                    f"Deactivated on {row['data_stop']}"
                ]),
                icon=custom_icon_red
            ) for _, row in inactive.iterrows()
        ]

        return [active_markers, inactive_markers]
    return [[],[]]

############################################# Callbacks DV_10 #############################################

@app.callback(
    Output("layers-control_dv_10", "children"),
    Input("el_map_dv_10", "n_clicks"),
    State("pollutant-dropdown_dv_10_11", "value")    
)
def update_layers_dv_10(n_clicks, selected_pollutant):
    if n_clicks == 0:
      return []
    
    if n_clicks > 0 and not selected_pollutant:
        return [
                dl.TileLayer(),
                dl.Popup(
                    position=[45.64, 9.60],  # Central position
                    children=html.Div(
                        "Select a pollutant to visualize the map",
                        style={"backgroundColor": "red", "color": "white", "padding": "10px", "borderRadius": "5px"}
                    ))
            ]
    
    gdf = get_data_10(selected_pollutant)
    unit = get_measurement_unit(selected_pollutant)
    colormap_c = cm.LinearColormap(["blue", "lightblue", "yellow", "orange", "red"], vmin=gdf['monthly_average'].min(), vmax=gdf['monthly_average'].max())
    gdf["color"] = gdf["monthly_average"].apply(lambda x: colormap_c(x)[:7])

    return [
        dl.Overlay(
                dl.GeoJSON(
                    data=geojson_data,  # Path to the GeoJSON file
                    zoomToBounds=True,
                    options={"style":style}
                 ),
                name="Contour lines",
                checked=True
            ),

            # Overlay dinamici per ogni anno
            *[
                dl.Overlay(
                    dl.LayerGroup([
                        dl.CircleMarker(
                            center=[row.geometry.y, row.geometry.x],
                            radius=5,
                            color=row["color"],
                            fill=False,
                            fillOpacity=0.8,
                            children=[dl.Tooltip(f"{row['month']}: {row['monthly_average']} {unit}")]
                        )
                        for _, row in gdf[gdf["month"].str.contains(str(year))].iterrows()
                    ]),
                    name=f"{year}",
                    checked=(year == get_years(gdf)[-2])  # default most recent year (-1) checked
                )
                for year in get_years(gdf)
            ]
    ]

############################################# Callbacks DV_11 #############################################

@app.callback(
    [Output('year-slider', 'min'),
     Output('year-slider', 'max'),
     Output('year-slider', 'value'),
     Output('year-slider', 'marks'),
     Output('month-dropdown', 'options'),
     Output('month-dropdown', 'value')],
    Input('pollutant-dropdown_dv_10_11', 'value')
)
def update_years_months(pollutant):
    if not pollutant:
        return (0, 0, 0, {}, [], 'not set')
    df = get_data_11(pollutant)
    years = get_years(df)
    months = get_months(df, years[0])

    return (float(years[0]), float(years[-1]), float(years[0]), {y: y for y in years},
            [{'label': m, 'value': m} for m in months], months[0]) #Default month set to ALL

@app.callback(
    Output('scatter-plot', 'figure'),
    Input("el_plot_dv_11", "n_clicks"),
    State('pollutant-dropdown_dv_10_11', 'value'), 
    State('year-slider', 'value'),
    State('month-dropdown', 'value')
)
def update_plot(n_clicks, pollutant, year, month):
    if n_clicks == 0:
      return px.scatter ()  # Return an empty figure if no button has been clicked
        
    if n_clicks > 0 and (not pollutant or not year or month is None):
        return px.scatter().update_layout(
            annotations=[dict(
                    text="Please select a pollutant, year and month to visualize the plot",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="white"),
                    bgcolor="red",
                    borderpad=5)])
    df = get_data_11(pollutant)
    unit = get_measurement_unit(pollutant)
    df_filtered = df[df["month"].str.contains(str(year))]

    if month != "All":
        df_selected = df_filtered[df_filtered["month"].str.startswith(month)]
        df_other = df_filtered[~df_filtered["month"].str.startswith(month)]
    else:
        df_selected = df_filtered
        df_other = pd.DataFrame()
    fig = px.scatter(
        df_selected,
        x='monthly_average',
        y='quota',
        color='monthly_average',
        color_continuous_scale='RdYlBu_r',
        labels={
            'monthly_average': f"Monthly Average per pollutant {unit}",
            'quota': "Height [m]"
        },
        title=f"Correlation between monthly average of {pollutant} and station height"
    ).update_traces(marker=dict(size=8))

    fig.update_coloraxes(showscale=False)  # Remove color bar
    if not df_other.empty:
        fig.add_scatter(
            x=df_other['monthly_average'],
            y=df_other['quota'],
            mode='markers',
            marker=dict(color='grey', opacity=0.2),
            name='Other months'
        )
    fig.update_layout(showlegend=False)
    return fig

############################################# Callbacks EU_DV_1 #############################################

# Callback to get the region of interest choosen by the user
@app.callback(
    Output("geojson_eu_dv_1", "data"),
    Input("edit_control_eu_dv_1", "geojson"))
def mirror(x):
    return x

# Callback to delete the region of interest choosen by the user
@app.callback(
    Output("edit_control_eu_dv_1", "editToolbar"), 
    Input("clear_all_eu_dv_1", "n_clicks"))
def trigger_action(n_clicks):
    return dict(mode="remove", action="clear all", n_clicks=n_clicks)  # include n_click to ensure prop changes

# Callback to populate the region of interest choosen by the user
@app.callback(
    [Output("active_eu_dv_1", "children"),
     Output("inactive_eu_dv_1","children")],
    Input("geojson_eu_dv_1", "data")
)
def get_station_inside_geometry(ROI):

    # If no ROI is selected, return empty lists
    if not ROI or "features" not in ROI or len(ROI["features"]) == 0:
        return [[], []]
    
    # Find the stations inside the ROI
    polygon_ROI = shape(ROI["features"][0]["geometry"])
    station = get_station_location()
    intersection = polygon_ROI.intersection(station.geometry)
    
    # Filters the stations that are inside the ROI
    new = station.copy()
    valid_indices = [i for i, geom in enumerate(intersection) if not geom.is_empty]
    new = new.iloc[valid_indices]

    if new.empty:
        return [[], []]

    # Gets all the sensors inside the filtered stations
    lista = new.id_stazione.tolist() #Converto to a list the stations inside the area
    list_sensors_inside_area=list_sensors_and_pollutants(lista) #seach for all sensors inside the area
    df = pd.DataFrame(list_sensors_inside_area) #created a df with all the sensors inside the area
    df['data_stop'] = pd.to_datetime(df['data_stop'], errors='coerce')
    df['data_stop'] = df['data_stop'].dt.strftime('%b %Y')

    # Create markers for the stations
    active_markers = []
    inactive_markers = []
    for index, row in new.iterrows():
        filtro = df[df['id_stazione'] == row['id_stazione'] ]
        filtro = filtro.drop('id_stazione', axis=1)
        if filtro["data_stop"].isna().any():
            # Crea Marker attivi
            active_markers.append(
                dl.Marker(
                    position=[row.geometry.y, row.geometry.x],
                    children=dl.Popup([
                        html.B(f"Dati rilevati nella stazione {row['id_stazione']} - {row['nome_stazione']}"),
                        html.Br(),
                        df_to_dash_table(filtro),
                    ]),
                    icon=custom_icon_green
                )
            )
        else:
            inactive_markers.append(
                dl.Marker(
                    position=[row.geometry.y, row.geometry.x],
                    children=dl.Popup([
                        html.B(f"Dati rilevati nella stazione {row['id_stazione']} - {row['nome_stazione']}"),
                        html.Br(),
                        df_to_dash_table(filtro),
                    ]),
                    icon=custom_icon_red
                )
            )
    return [active_markers, inactive_markers]

# Run the app in Jupyter notebook
print("Starting Dashboard...")
#app.run_server(port=8089)
app.run(port=8089, debug=True)

Starting Dashboard...
